In [2]:
import numpy as np
total = [0]
def my_matmult(m1, m2):
    ans_mat = np.empty((0,m2.shape[1]))
    for row in m1:
        ans_row = np.array([])
        for col in m2.T:
            ans_row = np.append(ans_row, np.dot(row, col))
            total[0] += len(row)*2
        ans_mat = np.vstack((ans_mat,ans_row))
    return ans_mat

a = np.random.rand(3, 2)
b = np.random.rand(2, 3)

print(np.array_equal(a@b, my_matmult(a,b)))


True


In [3]:
def tensor_product(m1, m2):
  ans = np.empty( (0, m1.shape[-2], m2.shape[-1]) )
  for i in range(m1.shape[0]):
    matmult = np.expand_dims(my_matmult(m1[i], m2[i]), axis=0)
    ans = np.vstack((ans, matmult))
  return ans

a = np.random.rand(2, 3, 4)
b = np.random.rand(2, 4, 2)

print(np.allclose(tensor_product(a,b), a @ b))


True


In [4]:
def nd_tensor_product(m1, m2):
  ans_shape = m1.shape[:-2] + (m1.shape[-2], m2.shape[-1])
  ans = np.empty(ans_shape)
  for i in range(m1.shape[0]):
    if len(m1.shape) == 3:
        ans[i] = my_matmult(m1[i], m2[i])

    else:
        ans[i] = nd_tensor_product(m1[i], m2[i])
  return ans

a = np.random.rand(12, 4, 64)
b = np.random.rand(12, 64, 4)

total = [0]

print(np.allclose(nd_tensor_product(a,b), a @ b))
print(total)

True
[24576]


In [6]:
import numpy as np
import bitsandbytes as bab

In [45]:
out = bnb.matmul(x, self.weight, bias=self.bias, state=self.state)


def my_matmul(m1, m2):
    m1_const = 2147483643/np.max(abs(m1))  #Squrt(127) is 11...
    m2_const = 2147483643/np.max(abs(m2))

    m1 = (m1 * m1_const).astype(np.int64)
    m2 = (m2* m2_const).astype(np.int64)

    # assert np.all((m1 >= -128) & (m1 <= 127))
    # assert np.all((m2 >= -128) & (m2 <= 127))

    ans = np.matmul(m1,m2)

    ans = (ans.astype(np.float64))/ (m1_const * m2_const)
    # ans = (ans / (m1_const * m2_const)).astype(np.float64)
    return ans

def nd_tensor_product(m1, m2):
    ans_shape = m1.shape[:-2] + (m1.shape[-2], m2.shape[-1])
    ans = np.empty(ans_shape)
    if len(ans_shape) < 3:
        return my_matmul(m1, m2)
    else:
        for i in range(ans_shape[0]):
            ans[i] = nd_tensor_product(m1[i], m2[i])
    return ans



# a = np.random.rand(4, 64)
# b = np.random.rand(64, 100)
# assert nd_tensor_product(a, b).shape == (4, 100)

# a = np.random.rand(12, 4, 64)
# b = np.random.rand(12, 64, 4)
# assert nd_tensor_product(a, b).shape == (12, 4, 4)

# a = np.random.rand(3, 12, 4, 64)
# b = np.random.rand(3, 12, 64, 4)
# assert nd_tensor_product(a, b).shape == (3,12, 4, 4)

# a = np.random.uniform(-100, 100, size=(2, 3))
# b = np.random.uniform(-100, 100, size=(3, 5))

a = np.array([[-30.03552628, -24.14292181, -58.09856082],
 [ 17.01992107, -16.35411441, -12.4997732 ]])

b = np.array([[ -4.27223891, -52.65948653,  10.15256336,  86.74645725,  41.73872511],
 [-70.12971761, -11.72562159,  28.39736422,  99.48703335,  53.35739328],
 [-37.84169407,  95.895102,    50.8691936,  -30.21012106,   0.36828462]])

# print(a)
# print(b)
print(np.matmul(a,b))
my_matmul(a,b)
# nd_tensor_product(a,b)


[[ 4020.00319733 -3706.6212591  -3945.95986582 -3252.21860792
  -2563.24475527]
 [ 1547.20884967 -1903.16517331  -927.47129922   227.01519137
   -166.82658161]]


array([[ 4020.00319268, -3706.62125742, -3945.9598619 , -3252.21860723,
        -2563.24475125],
       [ 1547.20884692, -1903.1651711 ,  -927.47129741,   227.01519176,
         -166.82658021]])